In [1]:
import pandas as pd

In [ ]:
d = pd.read_csv("../src/other/train_embedding_id.csv")

In [ ]:
d

In [ ]:
d.ad_idembedding = d.ad_idembedding.apply(eval)

In [ ]:
data = d.sample(frac = 1.0,random_state=2020)
#train = data.iloc[:850000,:]
train = data.iloc[:800000,:]
valid = data.iloc[800000:,:]

In [ ]:
mkdir -p /home/gezi/tmp/ad2020/tfrecords3

In [ ]:
mkdir -p /home/gezi/tmp/ad2020/tfrecords3/train

In [ ]:
mkdir -p /home/gezi/tmp/ad2020/tfrecords3/valid

In [3]:
import gezi
from gezi import tqdm
import melt
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
import os

In [ ]:
ofile = '/home/gezi/tmp/ad2020/tfrecords3/valid/record.TMP'
num_records = 0
with melt.tfrecords.Writer(ofile) as writer:
  for _, row in tqdm(valid.iterrows(), total=len(valid)):
    feature = {}
    ad_ids = list(map(int, row['ad_idembedding']))
    ad_ids = gezi.pad(ad_ids, 128)
    feature['age'] = melt.gen_feature([row['age']], np.int64)
    feature['gender'] = melt.gen_feature(row['gender'], np.int64)
    feature['ad_ids'] = melt.gen_feature(ad_ids, np.int64)
    record = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(record)
    num_records += 1

In [ ]:
ofile2 = ofile.replace('.TMP', f'.{num_records}')
os.system('mv %s %s' % (ofile, ofile2))

In [ ]:
!du -h /home/gezi/tmp/ad2020/tfrecords3/valid/record.100000

In [ ]:
ofile = '/home/gezi/tmp/ad2020/tfrecords3/train/record.TMP'
num_records = 0
with melt.tfrecords.Writer(ofile) as writer:
  for _, row in tqdm(train.iterrows(), total=len(train)):
    feature = {}
    ad_ids = list(map(int, row['ad_idembedding']))
    ad_ids = gezi.pad(ad_ids, 128)
    feature['age'] = melt.gen_feature([row['age']], np.int64)
    feature['gender'] = melt.gen_feature(row['gender'], np.int64)
    feature['ad_ids'] = melt.gen_feature(ad_ids, np.int64)
    record = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(record)
    num_records += 1

In [ ]:
ofile2 = ofile.replace('.TMP', f'.{num_records}')
os.system('mv %s %s' % (ofile, ofile2))

In [ ]:
!du -h /home/gezi/tmp/ad2020/tfrecords3/train/record.800000

In [10]:
from projects.ai.ad.src.dataset import Dataset
melt.init_flags()
FLAGS = melt.get_flags()
FLAGS.max_len = 128

In [11]:
dataset = Dataset('valid')

In [12]:
batches = dataset.make_batch(512, ['../input/tfrecords3/train/record.800000'])

In [14]:
i = 0
for item in batches:
  i += 1

In [15]:
print(i)

1563
